In [1]:
cd /content/drive/MyDrive/2022/dacon/구내식당식수인원예측경진대회

/content/drive/MyDrive/2022/dacon/구내식당식수인원예측경진대회


In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import io
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
import numpy as np
import seaborn as sns
import requests
import calendar
import datetime
from bs4 import BeautifulSoup

In [4]:
data = []
nowyear = 2021
nowmonth = 1

#### train 데이터셋을 위한 크롤링

#### 관측, 기후

In [6]:
from datetime import date, datetime, time
d = date(nowyear, nowmonth, 31)
d

datetime.date(2021, 1, 31)

In [ ]:
date, avg, max, min, rain = [], [], [], [], []

for year in range(2016,nowyear+1):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  for month in range(1, monthrange):
    print(year, month)
    response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=192&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 진주
    #response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 서울
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

    for a in table:
      if a.string:
        if a.string[-1] == '일':
          if month < 10:
            if int(a.string[:-1]) >= 10:
              date.append(str(year)+'-0'+str(month)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-0'+str(month)+'-0'+str(a.string[:-1]))

          else:
            if int(a.string[:-1]) >= 10:
              date.append(str(year)+'-'+str(month)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-'+str(month)+'-0'+str(a.string[:-1]))
        

        elif a.string.startswith('평균기온'):
          if len(a.string[5:-1]) != 0:
            avg.append(float(a.string[5:-1]))
          else:
            avg.append(None)
        elif a.string.startswith('최고기온'):
          max.append(float(a.string[5:-1]))
        elif a.string.startswith('최저기온'):
          min.append(float(a.string[5:-1]))
        elif a.string.startswith('일강수량'):
          if len(a.string[5:]) == 3:
            rain.append(0)
          else:
            rain.append(a.string[5:-2])

y : 2016  year : 2021
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
y : 2017  year : 2021
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
y : 2018  year : 2021
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
y : 2019  year : 2021
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
y : 2020  year : 2021
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
y : 2021  year : 2021
2021 1


In [ ]:
print(len(date),len(avg),len(min), len(max), len(rain))

1858 1858 1858 1858 1858


In [ ]:
weather = pd.DataFrame(date,columns=['date'])
weather['avg'] = avg
weather['max'] = max
weather['min'] = min
weather['rain'] = rain

In [ ]:
weather

,date,avg,max,min,rain
0,2016-01-01,-0.1,8.7,-6.7,0
1,2016-01-02,1.8,10.8,-3.6,0
2,2016-01-03,4.0,14.5,-2.2,0
3,2016-01-04,4.8,15.0,-0.3,0
4,2016-01-05,0.5,4.9,-2.6,0.0
...,...,...,...,...,...
1853,2021-01-27,4.1,11.4,-1.2,0
1854,2021-01-28,1.4,10.7,-3.0,0
1855,2021-01-29,-2.3,3.0,-6.0,0
1856,2021-01-30,-0.3,9.1,-9.2,0


In [ ]:
weather.to_csv('weather_jinju(최종수정).csv')

#### 상대 습도

In [ ]:
response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=192&yy=2018&obs=12') # 진주
#response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 서울
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

In [ ]:
date, humidity = [], []

for year in range(2016, nowyear+1):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  print(year)
  response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=192&yy='+str(year)+'&obs=12') # 진주
  # response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=108&yy='+str(year)+'&obs=12') # 진주

  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

  for a in table:
    if a.string:
      if a.string == '평균':
        break
      if a.string[-1] == '일':
        for i in range(1, 13):
          if i < 10:
            if int(a.string[:-1])>=10:
              date.append(str(year)+'-0'+str(i)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-0'+str(i)+'-0'+str(a.string[:-1]))
          else:
            if int(a.string[:-1])>=10:
              date.append(str(year)+'-'+str(i)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-'+str(i)+'-0'+str(a.string[:-1]))
          # date.append(str(year)+'-'+str(i)+'-'+str(a.string[:-1]))
      else:
        humidity.append(a.string)

y : 2016  year : 2021
2016
y : 2017  year : 2021
2017
y : 2018  year : 2021
2018
y : 2019  year : 2021
2019
y : 2020  year : 2021
2020
y : 2021  year : 2021
2021


In [ ]:
print(len(date), len(humidity))

2232 2232


In [ ]:
humidity_df = pd.DataFrame(date,columns=['date'])
humidity_df['humidity'] = humidity

In [ ]:
humidity_df

,date,humidity
0,2016-01-01,69.4
1,2016-02-01,43.9
2,2016-03-01,44.0
3,2016-04-01,57.3
4,2016-05-01,61.5
...,...,...
2227,2021-08-31,84.4
2228,2021-09-31,
2229,2021-10-31,80.4
2230,2021-11-31,


In [ ]:
humidity_df.to_csv('humidity_jinju(최종수정).csv')

### test 데이터셋을 위한 크롤링

#### 날씨

In [9]:
date, avg, max, min, rain = [], [], [], [], []

for year in range(2021,2022):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  for month in range(1, 5):
    print(year, month)
    response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=192&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 진주
    #response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 서울
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

    for a in table:
      if a.string:
        if a.string[-1] == '일':
          if month < 10:
            if int(a.string[:-1]) >= 10:
              date.append(str(year)+'-0'+str(month)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-0'+str(month)+'-0'+str(a.string[:-1]))

          else:
            if int(a.string[:-1]) >= 10:
              date.append(str(year)+'-'+str(month)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-'+str(month)+'-0'+str(a.string[:-1]))
        

        elif a.string.startswith('평균기온'):
          if len(a.string[5:-1]) != 0:
            avg.append(float(a.string[5:-1]))
          else:
            avg.append(None)
        elif a.string.startswith('최고기온'):
          max.append(float(a.string[5:-1]))
        elif a.string.startswith('최저기온'):
          min.append(float(a.string[5:-1]))
        elif a.string.startswith('일강수량'):
          if len(a.string[5:]) == 3:
            rain.append(0)
          else:
            rain.append(a.string[5:-2])

y : 2021  year : 2021
2021 1
2021 2
2021 3
2021 4


In [10]:
print(len(date),len(avg),len(min), len(max), len(rain))

120 120 120 120 120


In [11]:
weather = pd.DataFrame(date,columns=['date'])
weather['avg'] = avg
weather['max'] = max
weather['min'] = min
weather['rain'] = rain

In [12]:
weather.to_csv('weather_test.csv')

#### 습도

In [17]:
date, humidity = [], []

for year in range(2021, 2022):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  print(year)
  response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=192&yy='+str(year)+'&obs=12') # 진주
  # response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=108&yy='+str(year)+'&obs=12') # 진주

  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

  for a in table:
    if a.string:
      if a.string == '평균':
        break
      if a.string[-1] == '일':
        for i in range(1, 13):
          if i < 10:
            if int(a.string[:-1])>=10:
              date.append(str(year)+'-0'+str(i)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-0'+str(i)+'-0'+str(a.string[:-1]))
          else:
            if int(a.string[:-1])>=10:
              date.append(str(year)+'-'+str(i)+'-'+str(a.string[:-1]))
            else:
              date.append(str(year)+'-'+str(i)+'-0'+str(a.string[:-1]))
          # date.append(str(year)+'-'+str(i)+'-'+str(a.string[:-1]))
      else:
        humidity.append(a.string)

y : 2021  year : 2021
2021


In [18]:
print(len(date), len(humidity))

372 372


In [19]:
humidity_df = pd.DataFrame(date,columns=['date'])
humidity_df['humidity'] = humidity

In [20]:
humidity_df.to_csv('humidity_test.csv')

- ppt와 보고서에는 날씨 데이터는 기상청에서 크롤링했다 정도만 언급합시다
